4.1. Text Processing Setup

In [ ]:

import nltk
nltk . download ('stopwords')
nltk . download ('punkt')
nltk . download ('wordnet')
import os
import string
import logging
import re
from collections import defaultdict , Counter
from nltk . corpus import stopwords
from nltk . tokenize import word_tokenize
from nltk . stem import WordNetLemmatizer

STOPWORDS = set( stopwords . words ('english') )
LEMMATIZER = WordNetLemmatizer ()
import re

def preprocess(text):
    # Lowercasing and tokenizing the document
    tokens = re.findall(r'\w+', text.lower())
    return [token for token in tokens if token not in stop_words]

stop_words = {"the", "is", "and", "to", "in", "it"}  # Example stop words


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


4.2. Loading Documents


In [ ]:
documents = {
    1: "The first document is about Python programming.",
    2: "The second document discusses JavaScript and web development.",
    3: "Python is a popular programming language.",
    # Add more documents...
}


4.3. Inverted Index Construction

In [ ]:
from collections import defaultdict

def build_inverted_index(documents):
    inverted_index = defaultdict(set)
    for doc_id, text in documents.items():
        tokens = preprocess(text)
        for token in tokens:
            inverted_index[token].add(doc_id)
    return inverted_index

inverted_index = build_inverted_index(documents)


4.4. Boolean Retrieval

In [ ]:
# Function to handle Boolean queries (AND, OR, NOT)
def boolean_retrieval(query, inverted_index):
    query_terms = query.split()
    result = set()

    if "AND" in query_terms:
        terms = [term for term in query_terms if term != "AND"]
        if terms:
            result = set.intersection(*[inverted_index[term] for term in terms if term in inverted_index])

    elif "OR" in query_terms:
        terms = [term for term in query_terms if term != "OR"]
        # Check if there are any terms after filtering out "OR"
        if terms:
            # Create a set from the first term to use as the initial set for union
            result = inverted_index[terms[0]]
            # Perform union with the rest of the terms
            for term in terms[1:]:
                if term in inverted_index:
                    result = result.union(inverted_index[term])

    elif "NOT" in query_terms:
        not_term = query_terms[1]  # Expecting format like: NOT term
        if not_term in inverted_index:
            result = set(documents.keys()) - inverted_index[not_term]

    else:
        result = inverted_index[query] if query in inverted_index else set()

    return result

In [ ]:
# Building the inverted index from documents
inverted_index = build_inverted_index(documents)

4.5. Query Processing

In [ ]:
# Example queries
queries = [
    "Python AND programming",
    "JavaScript OR Python",
    "Python NOT JavaScript",
    "machine OR learning",
    "NOT JavaScript"
]

# Processing each query and displaying the result
for query in queries:
    result_docs = boolean_retrieval(query, inverted_index)
    print(f"Query: '{query}' -> Retrieved Documents: {result_docs}")

Query: 'Python AND programming' -> Retrieved Documents: {1, 3}
Query: 'JavaScript OR Python' -> Retrieved Documents: set()
Query: 'Python NOT JavaScript' -> Retrieved Documents: set()
Query: 'machine OR learning' -> Retrieved Documents: set()
Query: 'NOT JavaScript' -> Retrieved Documents: {1, 2, 3}
